In [39]:
import csv
import pandas as pd
import numpy as np
from IPython.display import display

In [40]:
target = 'scan/wheel-equities-1.csv'
balance = 100100
position_pp = 0.05

In [41]:
# open file
f = open('../' + target, 'r')
data = np.array(list(csv.reader(f)))
f.close()

In [42]:
# convert to dataframe
df = pd.DataFrame(data[:, 1:]).astype(np.float64)
df.index = data[:, 0]
df.columns = [
    'underlying', 'premium', 'dte', 'roc', 'be', 'be_moneyness', 
    'prob_be_delta', 'prob_be_iv', 'iv', 'iv_skew',
    'udl_year_ret', 'udl_year_ret_r2', 'udl_year_market_corr', 'udl_hist_vol', 
    'udl_iv_percentile', 'udl_hv_percentile', 'above_be_percentile'
]

In [43]:
# # strict filter dataframe
# df['annualized_roc'] = (1.0 + df['roc']) ** (365.2425 / df['dte']) - 1.0
# df = df[df['annualized_roc'] >= 0.020]
# df = df[df['prob_be_delta'] >= 0.80]
# df = df[df['prob_be_iv'] >= 0.80]

In [44]:
# refine columns
df_filt = pd.DataFrame().astype(np.float64)
df_filt['und'] = df['underlying']
df_filt['roc'] = df['roc']
df_filt['a_roc'] = (1.0 + df['roc']) ** (365.2425 / df['dte']) - 1.0
df_filt['be'] = df['be']
df_filt['score'] = df['prob_be_delta'] * df_filt['a_roc']
df_filt['be_moneyness'] = df['be_moneyness']
df_filt['prob_be_delta'] = df['prob_be_delta']
df_filt['prob_be_iv'] = df['prob_be_iv']
df_filt['iv_percentile'] = df['udl_iv_percentile']
df_filt['iv_skew'] = df['iv_skew']
df_filt['target_ask'] = df['premium'] / 100.0
df_filt['target_qty'] = np.floor((balance * position_pp) / (df['be'] * 100.0))
df_filt.index = df.index

In [45]:
# filter safe equities
safe_equities = [
    'AAL', 'UAL', 'AAPL', 'TWTR', 'SQ', 'SNAP', 'SHOP', 'ROKU', 
    'INTC', 'AMD', 'FB', 'CAT', 'AMZN', 'TSLA', 'T', 'CSCO', 
    'CVS', 'VZ', 'BAC', 'C', 'KO', 'TGT', 'PG', 'CLX', 'KMB', 
    'JNJ', 'TROW', 'F', 'WM', 'SYY', 'AFL', 'WFC', 'GE', 'DB', 'DIS',
    'NVDA'
]

top_indices = []
top_results = []
for equity in safe_equities:
    df_result = df_filt[df_filt.index.str.startswith(equity + ' ')]
    df_result = df_result.nlargest(1, 'score')
    if df_result.shape[0] == 1: 
        result = np.squeeze(df_result)
        top_indices.append(result.name)
        top_results.append(result)

top_results = pd.DataFrame(top_results)
top_results.columns = df_filt.columns
top_results.index = top_indices

In [46]:
# output results
top_results = top_results.sort_values('score', ascending=False)
pd.set_option('display.max_rows', 50)
display(top_results)

,und,roc,a_roc,be,score,be_moneyness,prob_be_delta,prob_be_iv,iv_percentile,iv_skew,target_ask,target_qty
AAL Nov 13 2020 $12.00 Put,12.33,0.09489,1.909804,10.96,1.392324,0.88889,0.72904,0.67115,0.27149,0.96788,1.04,4.0
TSLA Nov 13 2020 $445.00 Put,446.30,0.09700,1.976563,405.65,1.385867,0.90892,0.70115,0.65477,0.72398,0.99595,39.35,0.0
SNAP Nov 13 2020 $27.00 Put,27.09,0.08521,1.620740,24.88,1.139267,0.91842,0.70293,0.65779,0.31222,0.99524,2.12,2.0
ROKU Nov 20 2020 $235.00 Put,235.37,0.10510,1.613127,212.65,1.099782,0.90347,0.68177,0.65777,0.50679,0.99761,22.35,0.0
UAL Nov 20 2020 $35.00 Put,35.40,0.09718,1.438578,31.90,1.028713,0.90113,0.71509,0.66064,0.30769,1.01001,3.10,1.0
SQ Nov 20 2020 $190.00 Put,190.10,0.09290,1.348666,173.85,0.951619,0.91452,0.70560,0.65571,0.58824,0.99638,16.15,0.0
AMD Nov 13 2020 $85.00 Put,85.70,0.07459,1.334012,79.10,0.908769,0.92299,0.68123,0.66433,0.52489,0.99263,5.90,0.0
TWTR Nov 20 2020 $47.00 Put,47.14,0.08170,1.127271,43.45,0.795774,0.92172,0.70593,0.65585,0.45701,0.99665,3.55,1.0
SHOP Nov 20 2020 $1100.00 Put,1102.74,0.08151,1.123682,1017.10,0.781262,0.92234,0.69527,0.65774,0.61086,1.00144,82.90,0.0
AAPL Nov 13 2020 $121.00 Put,121.37,0.05815,0.946328,114.35,0.646285,0.94216,0.68294,0.66241,0.79638,0.98628,6.65,0.0
